# Dataset downloaden en setup
Dataset-link: https://github.com/adobe-research/VideoSham-dataset

## Dataset downloaden:
### stap 1:
- >Volg de Dataset-link naar de gihub pagina en scroll naar beneden en download "Unedited-Part1.zip" onder "Dataset Structure"

### stap 2:
- >extract de gedownloadde "Unedited-Part1.zip" naar een folder in je projectfolder genaamd "Videos"

## setup
Run alle cellen tot de volgende markdown cel om de setup te voltooien.

In [3]:
import cv2
import os
import moviepy.editor as mp
import pandas as pd

In [4]:
base_path = 'Videos'
resized_path = 'resized_' + base_path 

In [5]:
def get_video_paths(path):
    video_paths = []
    for video in os.listdir(path):
        if video.endswith('.mp4'):
            video_paths.append(f'{path}/{video}')
        else:
            pass
    return video_paths

In [6]:
video_paths = get_video_paths(base_path)

In [7]:
def resize_video(folder, paths):
    try:
        os.mkdir(folder)
    except FileExistsError:
        print(f'{folder} exists, no folder creation needed!')
    for index, path in enumerate(paths):
        new_path = f'{folder}/video_{index}.mp4'
        clip = mp.VideoFileClip(path)
        clip_resized = clip.resize((1280, 720))
        clip_resized.write_videofile(new_path)

## Resizing videos
run dit door de volgende code cel te uncommenten
### run dit als:
- >Dit de eerste keer is dat je dit project runt

- >De geresizede images per ongeluk veranderd of verwijderd zijn

In [8]:
# resize_video(resized_path, video_paths)

## Maak een imagePainter class aan:
Hier wordt een imagePainter class aangemaakt die wordt gebruikt om de frames(images) van een video te labelen. Je heeft een video_path mee bij het aanmaken van het object en dan runt de labeler automatisch met de aangegeven video. 

In [11]:
class imagePainter:
    def __init__(self, video_path, title='ImagePainter!'):
        self.running = True
        
        self.title = title
        self.video_name = video_path.lstrip('resized_Videos/').rstrip('.mp4')
        self.rectangle_coords = pd.DataFrame({'image': [], 'x1': [], 'y1': [], 'x2': [], 'y2': []})
        
        self.frame_folder, self.face_folder, self.data_folder = self.create_folders()
        
        self.draw_on_frames_from_video(video_path)
        
    def create_folders(self):
        data_folder = 'data'
        frame_folder = f'{data_folder}/{self.video_name}_frames'
        face_folder =  f'{frame_folder}_face_boxes'

        self.check_and_create_folder(data_folder)
        self.check_and_create_folder(frame_folder)
        self.check_and_create_folder(face_folder)
        
        return (frame_folder, face_folder, data_folder)
    
    def check_and_create_folder(self, folder):
        try:
            os.mkdir(folder)
            print(f'Creating {folder}')
        except FileExistsError:
            print(f'{folder} folder already exists! skipping creation...')
            
    def get_data_paths(self, count):
        frame_path = f'{self.frame_folder}/{self.video_name}_frame_{count}.jpg'
        face_path = f'{self.face_folder}/{self.video_name}_face_frame_{count}.jpg'
        self.coords_path = f'{self.data_folder}/{self.video_name}_coords.csv'
        
        return (frame_path, face_path)
    

                
    def draw_on_frames_from_video(self, video_path):
        self.show_previous_frame = False
        vid = cv2.VideoCapture(video_path)
        frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        count = 0
        
        while self.running and vid.isOpened():
            if self.show_previous_frame:
                count -= 1
                vid.set(cv2.CAP_PROP_POS_FRAMES, current_frame-1)
                self.running, self.img = vid.read()
                self.show_previous_frame = False
            else:
                self.running, self.img = vid.read()
                
                
            next_frame = vid.get(cv2.CAP_PROP_POS_FRAMES)
            current_frame = next_frame - 1
            
            try:
                self.original_img = self.img.copy()
            except AttributeError:
                cv2.destroyAllWindows() 
                self.rectangle_coords.to_csv(self.coords_path, index=False)
                break
            self.drawing_img = self.img.copy()

            self.image_painter(count)
            
            count += 1
            self.title = f'ImagePainter - {self.video_name} - Image{count}/{frame_count}'  
        self.rectangle_coords.to_csv(self.coords_path, index=False)
    def image_painter(self, count):
        self.image_name = f'{self.video_name}_frame_{count}.jpg'
        frame_path, face_path = self.get_data_paths(count)
        
        self.ix = -1
        self.iy = -1
        self.drawing = False
        
        while self.running:
            cv2.namedWindow(winname = self.title) 
            cv2.setMouseCallback(self.title,  self.draw_rectangle_with_drag) 
            cv2.imshow(self.title, self.img)
                
            key = cv2.waitKey(0)
            if key == 27:
                self.running = False
                cv2.destroyAllWindows() 
                self.rectangle_coords.to_csv(self.coords_path, index=False)
            elif key == ord('z'):
                self.show_previous_frame = True
                print('Going to the previous frame')
                cv2.destroyAllWindows() 
                break
            else:
                cv2.imwrite(frame_path, self.original_img)
                cv2.imwrite(face_path, self.drawing_img)
                cv2.destroyAllWindows()  
                break
    
    def draw_rectangle_with_drag(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN: 
            self.drawing = True
            self.ix = x 
            self.iy = y
            
        elif event == cv2.EVENT_MOUSEMOVE: 
            if self.drawing == True: 
                self.drawing_img = self.img.copy()
                cv2.rectangle(self.drawing_img, 
                              pt1=(self.ix, self.iy), pt2=(x, y), 
                              color=(0, 0, 255), 
                              thickness=1, 
                              lineType=cv2.LINE_AA)
                cv2.imshow(self.title, self.drawing_img)
        
        elif event == cv2.EVENT_LBUTTONUP: 
            self.img = self.drawing_img
            coords = pd.DataFrame({'image': [self.image_name], 'x1': [self.ix], 'y1': [self.iy], 'x2': [x], 'y2': [y]})
            self.rectangle_coords = pd.concat([self.rectangle_coords, coords], axis=0)
            self.drawing = False
            

# Run de ImagePainter:
Hieronder wordt rezised_video_paths gedefinieerd, deze variabel bevat een lijst met de paths naar alle geresizede videos

Kies een video door de video variabele gelijk te zetten aan een waarde uit de lijst met videos.

<b><u> Houd bij welke videos je hebt gelabeled om dubbel werk te voorkomen!!!</u></b>

# imagePainter uitleg:
De imagePainter opent een frame van de video en laat deze aan jou zien in een aparte window, op deze frame kan jij rechthoeken tekenen door je linkermuisknop in te drukken en te slepen.
Zodra je alle gezichten heb gelabeld kan je op een willekeurige knop drukken zolang deze nog niet wordt gebruikt voor een andere doeleinde.

## knoppen met andere doeleindes:
- escape: 
    - eindig het programma en sla de coordinaten van alle labels op
- z:
    - Ga een frame terug en label deze opnieuw
    - Als je op de eerste frame bent wordt het frame gerefreshed

<b>Als alle frames van een video gelabeld zijn eindigt het programma ook.</b>

In [12]:
rezised_video_paths = get_video_paths(resized_path)
video = rezised_video_paths[0]
image_painter = imagePainter(video)

data folder already exists! skipping creation...
data/video_0_frames folder already exists! skipping creation...
data/video_0_frames_face_boxes folder already exists! skipping creation...
